In [1]:
!pip install bitsandbytes==0.40.2 peft trl==0.4.7 mlflow accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
!git clone https://github.com/harsh19/Shakespearizing-Modern-English.git data

Cloning into 'data'...
remote: Enumerating objects: 140, done.
remote: Total 140 (delta 0), reused 0 (delta 0), pack-reused 140
Receiving objects: 100% (140/140), 247.18 MiB | 18.42 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Updating files: 100% (38/38), done.


In [3]:
import requests
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pickle
import random
import numpy as np
import torch
from torch.utils.data import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import TrainingArguments
from datasets import load_dataset
import json
from datasets import Dataset
import mlflow
from trl import SFTTrainer
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import re
from tqdm import tqdm

In [4]:
def load_file(path):
  data = []
  data_file = open("data/data/" + path)
  for line in data_file:
    data.append(line.split('\n')[0])
  return data

In [18]:
def load_dataset(mode):
  template = """{} Rewrite that sentence to sound like Shakespeare victorian English: """
  train_modern = load_file('{}.{}.nltktok'.format(mode, 'modern'))
  train_original = load_file('{}.{}.nltktok'.format(mode, 'original'))
  train_df = pd.DataFrame({'prompt':train_modern, 'response':train_original})
  train_df['prompt'] = train_df["prompt"].apply(lambda x: template.format(x))
  train_df['response'] =  train_df['response']
  train_df = train_df[['prompt', 'response']]
  if( mode == 'train') or (mode == 'valid'):
    train_df['text'] = train_df["prompt"] + train_df["response"]
    train_df.drop(columns=['prompt', 'response'], inplace=True)
  return train_df

In [19]:
train_dataset = load_dataset('train')
validation_dataset = load_dataset('valid')

In [20]:
print("==> Train Dataset Example:\n", train_dataset.iloc[0]['text'])
print()
print("==> Validation Dataset Example:\n", validation_dataset.iloc[0]['text'])

==> Train Dataset Example:
 I have half a mind to hit you before you speak again . Rewrite that sentence to sound like Shakespeare victorian English: I have a mind to strike thee ere thou speak'st .

==> Validation Dataset Example:
 Now , you lie there on the path . Rewrite that sentence to sound like Shakespeare victorian English: Lie thou there ( throwing down a letter ) , for here comes the trout that must be caught with tickling .


In [8]:
train_dataset = Dataset.from_pandas(train_dataset)
validation_dataset = Dataset.from_pandas(validation_dataset)

In [9]:
checkpoint = "bigscience/mt0-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto")

config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [21]:
test_dataset = load_dataset('test')

In [22]:
test_dataset.head()

,prompt,response
0,A jumbled confession can only receive a jumble...,Riddling confession finds but riddling shrift .
1,I love rich Capulet's daughter . Rewrite that ...,Then plainly know my heart's dear love is set ...
2,We're bound to each other in every possible wa...,"As mine on hers , so hers is set on mine , And..."
3,I'll tell you more later about when and where ...,"When and where and how We met , we wooed and m..."
4,"Holy Saint Francis , this is a drastic change ...","Holy Saint Francis , what a change is here !"


## Zero Shot

In [23]:
train_name = 'zero_shot_finetune'

In [25]:
output_data = []
for index, row in tqdm(test_dataset.iterrows()):
  prompt = row['prompt']
  expected_response = row['response']
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
  generation_output = model.generate(input_ids=input_ids, max_length=100)
  model_response = tokenizer.decode(generation_output[0], skip_special_tokens=True)
  output_data.append([model_response, expected_response])

1462it [08:03,  3.02it/s]


In [26]:
output_data = np.array(output_data)

In [27]:
test_df = pd.DataFrame({"model_response": output_data[:, 0], "expected_response":output_data[:, 1]})
test_df.to_pickle(f"./{train_name}.pkl")

## Few Shot

In [28]:
train_name = 'few_shot_finetune'

In [29]:
few_shot_prompt = ""
for index, row in tqdm(train_dataset.iterrows()):
  few_shot_prompt += row['text'] + "\n\n"
  if (index >= 5):
    break

print(few_shot_prompt)

5it [00:00, 2244.62it/s]

I have half a mind to hit you before you speak again . Rewrite that sentence to sound like Shakespeare victorian English: I have a mind to strike thee ere thou speak'st .

But if Antony is alive , healthy , friendly with Caesar , and not Caesar's prisoner , I'll shower you with gold and pearls . Rewrite that sentence to sound like Shakespeare victorian English: Yet if thou say Antony lives , is well , Or friends with Caesar , or not captive to him , I'll set thee in a shower of gold and hail Rich pearls upon thee .

Madam , he's well . Rewrite that sentence to sound like Shakespeare victorian English: Madam , he's well .

That's well spoken . Rewrite that sentence to sound like Shakespeare victorian English: Well said .

And he's friendly with Caesar . Rewrite that sentence to sound like Shakespeare victorian English: And friends with Caesar .

You are an honest man . Rewrite that sentence to sound like Shakespeare victorian English: Th' art an honest man .




In [36]:
output_data = []
for index, row in tqdm(test_dataset.iterrows()):
  prompt = few_shot_prompt + row['prompt']
  expected_response = row['response']
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
  generation_output = model.generate(input_ids=input_ids, max_length=300)
  model_response = tokenizer.decode(generation_output[0], skip_special_tokens=True)
  output_data.append([model_response, expected_response])

1462it [17:09,  1.42it/s]


In [37]:
output_data = np.array(output_data)

In [40]:
test_df = pd.DataFrame({"model_response": output_data[:, 0], "expected_response":output_data[:, 1]})
test_df.to_pickle(f"./{train_name}.pkl")

In [34]:
train_name

'few_shot_finetune'